# Module 8 Lab Assignment

In [ ]:
#import pandas and numpy
import pandas as pd
import numpy as np

In this lab, you will work on a dataset about students' video activities.

This dataset includes the counts about specific students' actions (such as play, pause, etc.) on different videos.

Import the data file (`video-activities.xlsx`) into a dataframe called `video_data` and display the first 5 rows:

In [ ]:
video_data = pd.read_excel('video-activities.xlsx')
video_data.head()

,StudentId,Seconds,Action,VideoId
0,1.0,15.0,play,1.0
1,1.0,37.0,faster,1.0
2,1.0,70.0,caption,1.0
3,1.0,87.0,slower,1.0
4,1.0,142.0,forward,1.0


Print the dimensions of `video_data`.  You can use `shape` attribute.

In [ ]:
print(video_data.shape)

(250, 4)


**TASK 1 [5 PTS]** Sort the data by `StudentId` and `VideoId` columns in an ascending order.

In [ ]:
sorted_data = video_data.sort_values(by=['StudentId', 'VideoId'])
sorted_data.head()

,StudentId,Seconds,Action,VideoId
0,1.0,15.0,play,1.0
1,1.0,37.0,faster,1.0
2,1.0,70.0,caption,1.0
3,1.0,87.0,slower,1.0
4,1.0,142.0,forward,1.0


### Cleaning Data

**TASK 2 [5 PTS]** There might be some missing values in the data. Use the `isna` and `sum` functions together to print the count of NA (missing values) for each column.

In [ ]:
count_na = video_data.isna().sum()
print(count_na)

StudentId    3
Seconds      2
Action       1
VideoId      3
dtype: int64


**TASK 3 [5 PTS]** Then, drop the records where `StudentId` is missing. After the drop, please print the shape of the dataframe.

In [ ]:
video_data.dropna(subset=['StudentId'], inplace=True)
display(video_data.shape)

(247, 4)

**TASK 4 [10 PTS]** Next, please drop the rows where there are two or more missing values among the `Seconds`, `Action`, `VideoId` columns.

In [ ]:
columns_to_check = ['Seconds', 'Action', 'VideoId']
video_data.dropna(subset=columns_to_check, thresh=len(columns_to_check) - 1, inplace=True)
display(video_data.shape)

(246, 4)

**TASK 5 [5 PTS]** There might be still missing values left in the `Seconds`, `Action`, `VideoId` columns. Replace these values using forward fill method.

In [ ]:
columns_to_fill = ['Seconds', 'Action', 'VideoId']
video_data[columns_to_fill] = video_data[columns_to_fill].fillna(method='ffill')

/tmp/ipython-input-2704700003.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  video_data[columns_to_fill] = video_data[columns_to_fill].fillna(method='ffill')


Once more, please display the NA counts for each column. Right now only the `Count` column should have 10 missing values.

In [ ]:
count_na = video_data.isna().sum()
print(count_na)

StudentId    0
Seconds      0
Action       0
VideoId      0
dtype: int64


**TASK 6 [5 PTS]** Before moving on, please change the StudentId, Seconds and VideoId columns to int32:

*Do you know why were they stored as float after imported with Pandas? Whoever first answers this in Slack #general channel will get +1 bonus*

In [ ]:
video_data['StudentId'] = video_data['StudentId'].astype('int32')
video_data['Seconds'] = video_data['Seconds'].astype('int32')

### Merging DataFrames

Import the `video-module-match.xlsx` table into a dataframe called `video_modules`.

Then print the first 5 rows of `video_modules`.

This data indicates to which module of the course the videos belong to.

In [ ]:
video_modules = pd.read_excel('video-module-match.xlsx')
video_modules.head()

,VideoId,ModuleId
0,1,1
1,2,1
2,3,2
3,1,1
4,1,1


**TASK 7 [5 PTS]** You might have noticed that there are many duplicate values in `video_modules` dataset. Please drop the duplicate rows:

In [ ]:
video_modules.drop_duplicates(inplace=True)
display(video_modules)

,VideoId,ModuleId
0,1,1
1,2,1
2,3,2


**TASK 8 [10 PTS]** Use `merge` to join `video_data` with `video_modules`. The merge operation should keep all records from `video_data`, and should just bring the matching records from `video_modules`.

The resulting dataframe of join should be stored into `video_data_comp`.

In [ ]:
video_data_comp = pd.merge(video_data, video_modules, on='VideoId', how='left')
video_data_comp.head()

,StudentId,Seconds,Action,VideoId,ModuleId
0,1,15,play,1.0,1
1,1,37,faster,1.0,1
2,1,70,caption,1.0,1
3,1,87,slower,1.0,1
4,1,142,forward,1.0,1


### Multi-level Indexing

**TASK 9 [5 PTS]** Create a multi-level indexing on the `video_data_comp` dataset as shown in the following output:

In [ ]:
video_data_comp.set_index(['StudentId', 'VideoId'], inplace=True)
video_data_comp

Seconds    Action  ModuleId
StudentId VideoId                             
1         1.0           15      play         1
          1.0           37    faster         1
          1.0           70   caption         1
          1.0           87    slower         1
          1.0          142   forward         1
...                    ...       ...       ...
10        3.0          168     pause         2
          3.0          208  backward         2
          3.0          240     pause         2
          3.0          265    slower         2
          3.0          293     pause         2

[246 rows x 3 columns]

**TASK 10 [5 PTS]** Change the order of the row indices, and sort them by `Day` in increasing order as shown below:

In [ ]:
video_data_comp = video_data_comp.swaplevel('StudentId', 'VideoId')
video_data_comp.sort_index(level='VideoId', ascending=True, inplace=True)
video_data_comp

Seconds    Action  ModuleId
VideoId StudentId                             
1.0     1               15      play         1
        1               37    faster         1
        1               70   caption         1
        1               87    slower         1
        1              142   forward         1
...                    ...       ...       ...
3.0     10             168     pause         2
        10             208  backward         2
        10             240     pause         2
        10             265    slower         2
        10             293     pause         2

[246 rows x 3 columns]

**TASK 11 [5 PTS]** Apply proper slicing to row indices to display the `Actions` of the student with id #1 on videos 1 and 2:

In [ ]:
video_data_comp.loc[(pd.IndexSlice[[1.0, 2.0]], 1), 'Action']

VideoId  StudentId
1.0      1                play
         1              faster
         1             caption
         1              slower
         1             forward
         1               pause
2.0      1                play
         1             forward
         1               pause
         1             forward
         1                play
         1             caption
         1               pause
         1            backward
Name: Action, dtype: object

**TASK 12 [5 PTS]** Apply proper slicing to show all activities of student with id 2:

In [ ]:
video_data_comp.loc[pd.IndexSlice[:, 2], :]

Seconds    Action  ModuleId
VideoId StudentId                             
1.0     2                7      play         1
        2               30   caption         1
        2               55      play         1
        2               88   caption         1
        2              114   forward         1
        2              136      play         1
        2              160    slower         1
2.0     2                1  backward         1
        2               47  backward         1
        2               61   forward         1
        2              106     pause         1
3.0     2                6  backward         2
        2               21    slower         2
        2               58     muted         2
        2              115   unmuted         2
        2              172      play         2
        2              192   unmuted         2
        2              242     pause         2
        2              271    slower         2
        2              298      play         2
        2              348   unmuted         2

**TASK 13 [5 PTS]** Change the index of the dataframe as shown below:

In [ ]:
video_data_comp = video_data_comp.reset_index()
video_data_comp.set_index('StudentId', inplace=True)
video_data_comp

,VideoId,Seconds,Action,ModuleId
StudentId,,,,
1,1.0,15,play,1
1,1.0,37,faster,1
1,1.0,70,caption,1
1,1.0,87,slower,1
1,1.0,142,forward,1
...,...,...,...,...
10,3.0,168,pause,2
10,3.0,208,backward,2
10,3.0,240,pause,2


**TASK 14 [5 PTS]** Create a dummy variable from the `Action` column (including only play, pause, backward, and forward). Only consider the actions correspondong to Module 1.

In [ ]:
module1_data = video_data_comp[video_data_comp['ModuleId'] == 1]

actions_to_dummy = ['play', 'pause', 'backward', 'forward']

m1Dummies = (pd.get_dummies(module1_data['Action']).reindex(columns=actions_to_dummy, fill_value=0).astype(int))

m1Dummies

,play,pause,backward,forward
StudentId,,,,
1,1,0,0,0
1,0,0,0,0
1,0,0,0,0
1,0,0,0,0
1,0,0,0,1
...,...,...,...,...
10,0,0,0,0
10,0,0,0,0
10,0,0,0,0


The following code is provided to compute the total number of each action per each student. This code uses `groupby` which will be covered next week.

In [ ]:
module1Actions = m1Dummies.groupby(level=0).sum()
module1Actions

,play,pause,backward,forward
StudentId,,,,
1,3,3,1,3
2,3,1,2,2
3,2,2,3,0
4,1,1,1,4
5,0,2,1,1
6,2,2,2,0
7,2,2,4,3
8,1,2,1,0
9,0,3,1,1


**TASK 15 [5 PTS]** Create a dummy variable from the `Action` column (including only play, pause, backward, and forward). Only consider the actions correspondong to Module 2.

,play,pause,backward,forward
StudentId,,,,
1,1,0,0,0
1,0,0,0,0
1,0,0,1,0
1,0,1,0,0
1,1,0,0,0
...,...,...,...,...
10,0,1,0,0
10,0,0,1,0
10,0,1,0,0


The following code is provided to compute the total number of each action per each student. This code uses `groupby` which will be covered next week.

In [ ]:
module2Actions = m2Dummies.groupby(level=0).sum()
module2Actions

,play,pause,backward,forward
StudentId,,,,
1,2,2,2,1
2,2,1,1,0
3,2,0,0,0
4,1,0,1,0
5,2,1,1,2
6,0,0,0,3
8,2,0,1,2
9,2,3,4,2
10,0,4,1,1


**TASK 16 [5 PTS]** Join  `module1Actions` and `module2Actions` to obtain the dataframe shown below:

In [ ]:
merged_actions = pd.concat([module1Actions, module2Actions], axis=1, keys=['M1', 'M2'], join = "inner")
merged_actions

M1                          M2                       
          play pause backward forward play pause backward forward
StudentId                                                        
1            3     3        1       3    2     2        2       1
2            3     1        2       2    2     1        1       0
3            2     2        3       0    2     0        0       0
4            1     1        1       4    1     0        1       0
5            0     2        1       1    2     1        1       2
6            2     2        2       0    0     0        0       3
8            1     2        1       0    2     0        1       2
9            0     3        1       1    2     3        4       2
10           3     0        0       1    0     4        1       1

**TASK 17 [5 PTS]** From the dataframe you obtain in the previous task, please get the total numbers of play and pause events for M1.

In [ ]:
m1_play_pause_totals = merged_actions['M1'][['play', 'pause']].sum()
print(m1_play_pause_totals)

play     17
pause    18
dtype: int64


**TASK 18 [5 PTS]** From the dataframe you obtain in the previous task, please get the total numbers of play and pause events for M2.

In [ ]:
m2_play_pause_totals = merged_actions['M2'][['play', 'pause']].sum()
print(m2_play_pause_totals)

play     13
pause    11
dtype: int64


StudentId
1     5.0
2     3.0
3     2.0
4     1.0
5     3.0
6     0.0
7     0.0
8     2.0
9     5.0
10    4.0
dtype: float64

**TASK 19 [5 PTS]** Identify students whose total count of play and pause events is higher in M2 compared to M1.

In [ ]:
total_play_pause_m1_per_student = module1Actions[['play', 'pause']].sum(axis=1)
total_play_pause_m2_per_student = module2Actions[['play', 'pause']].sum(axis=1)

total_play_pause_m2_per_student_aligned = total_play_pause_m2_per_student.reindex(total_play_pause_m1_per_student.index, fill_value=0)

students_m2_higher = total_play_pause_m2_per_student_aligned[total_play_pause_m2_per_student_aligned > total_play_pause_m1_per_student].index

print(students_m2_higher)

Index([5, 9, 10], dtype='int32', name='StudentId')
